In [1]:
import numpy as np
import pandas as pd
import math
from sympy import primerange
from scipy.stats import entropy

# Constants and derivations
phi = (1 + math.sqrt(5)) / 2
pi = math.pi
sqrt2 = math.sqrt(2)
alpha = 1 / 137.035999
alpha_inv = 137.035999
e = math.e

# High-resolution folding bases and powers
fold_base = 9
fold_powers = [1, 2, 3, 4, 5]

# Digit signatures to search for symbolic presence
digit_signatures = [
    "7297", "36034", "297", "035", "137035", "1137", "000137", "734", "134", "729735", "1370", "730"
]

# Key constants
target_constants = {
    "alpha": alpha,
    "alpha_inv": alpha_inv,
    "pi": pi,
    "phi": phi,
    "sqrt2": sqrt2,
    "e": e,
}

# Fibonacci word generation
def fibonacci_word(n):
    a, b = "0", "01"
    while len(b) < n:
        a, b = b, b + a
    return b[:n]

# Convert bitstring to decimal
def fibword_to_decimal(s):
    return sum(int(b) / 2 ** (i + 1) for i, b in enumerate(s))

# Decimal to base-n converter
def decimal_to_base(n, base, length=14):
    digits = []
    for _ in range(length):
        n *= base
        digit = int(n)
        digits.append(str(digit))
        n -= digit
    return "".join(digits)

# Entropy of digit string
def digit_entropy(s):
    counts = [s.count(str(d)) for d in range(10)]
    freqs = [c / len(s) for c in counts if c > 0]
    return -sum(f * math.log(f, 2) for f in freqs) if freqs else 0

# Substring signature match count
def match_digit_signatures(s):
    return sum(1 for sig in digit_signatures if sig in s)

# New test runner
records = []
fib_sequence = fibonacci_word(300)

for i in range(20, len(fib_sequence) - 14):
    bit_window = fib_sequence[i:i+14]
    dec_val = fibword_to_decimal(bit_window)

    row = {
        "bit_window": bit_window,
        "decimal_val": dec_val
    }

    for name, const in target_constants.items():
        delta = abs(dec_val - const)
        row[f"delta_{name}"] = delta

        for p in fold_powers:
            folded = delta / (fold_base ** p)
            row[f"delta_{name}_fold9^{p}"] = folded

    for base in [2, 3, 9, 10]:
        base_repr = decimal_to_base(dec_val, base)
        row[f"base{base}_repr"] = base_repr
        row[f"entropy_base{base}"] = digit_entropy(base_repr)
        row[f"sig_match_base{base}"] = match_digit_signatures(base_repr)

    records.append(row)

df_stringent_zeta = pd.DataFrame(records)

from IPython.display import display
display(df_stringent_zeta)


,bit_window,decimal_val,delta_alpha,delta_alpha_fold9^1,delta_alpha_fold9^2,delta_alpha_fold9^3,delta_alpha_fold9^4,delta_alpha_fold9^5,delta_alpha_inv,delta_alpha_inv_fold9^1,...,sig_match_base2,base3_repr,entropy_base3,sig_match_base3,base9_repr,entropy_base9,sig_match_base9,base10_repr,entropy_base10,sig_match_base10
0,00100101001001,0.145081,0.137783,0.015309,0.001701,0.000189,0.000021,0.000002,136.890918,15.210102,...,0,01022020212120,1.530619,0,12667767587445,2.610577,0,14508056640625,2.645593,0
1,01001010010010,0.290161,0.282864,0.031429,0.003492,0.000388,0.000043,0.000005,136.745838,15.193982,...,0,02121111202011,1.492614,0,25446646286002,2.413800,0,29016113281250,2.753434,0
2,10010100100101,0.580383,0.573086,0.063676,0.007075,0.000786,0.000087,0.000010,136.455616,15.161735,...,0,12020000220011,1.492614,0,52008043016785,2.950212,0,58038330078125,2.645593,0
3,00101001001010,0.160767,0.153469,0.017052,0.001895,0.000211,0.000023,0.000003,136.875232,15.208359,...,0,01110001210022,1.530619,0,14017086034682,2.896292,0,16076660156250,2.352746,0
4,01010010010100,0.321533,0.314236,0.034915,0.003879,0.000431,0.000048,0.000005,136.714466,15.190496,...,0,02220010120122,1.530619,0,28035183070475,2.896292,1,32153320312500,2.270942,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,10010100100101,0.580383,0.573086,0.063676,0.007075,0.000786,0.000087,0.000010,136.455616,15.161735,...,0,12020000220011,1.492614,0,52008043016785,2.950212,0,58038330078125,2.645593,0
262,00101001001010,0.160767,0.153469,0.017052,0.001895,0.000211,0.000023,0.000003,136.875232,15.208359,...,0,01110001210022,1.530619,0,14017086034682,2.896292,0,16076660156250,2.352746,0
263,01010010010100,0.321533,0.314236,0.034915,0.003879,0.000431,0.000048,0.000005,136.714466,15.190496,...,0,02220010120122,1.530619,0,28035183070475,2.896292,1,32153320312500,2.270942,0
264,10100100101001,0.643127,0.635830,0.070648,0.007850,0.000872,0.000097,0.000011,136.392872,15.154764,...,0,12210021120002,1.577406,0,57075025474851,2.610577,0,64312744140625,2.807355,0
